### The Evolution of Lewis Hamilton: A data driven career overview

In [3]:
#import libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import fastf1

In [ ]:
#set up a cache
fastf1.Cache.enable_cache('cache') 

In [ ]:
#getting names of circuits
years = list(range(2018,2025))

circuits = set()

for year in years:
    try:
        schedule = fastf1.get_event_schedule(year)
        event_names = schedule['EventName'].tolist()
        circuits.update(event_names)
    except Exception as e:
        print(f"Error fetching data from {year}: {e}")

print("Circuits from 2018-2024:")
for event in sorted(circuits):
    print(f"{event}")

In [ ]:
#extracting race data
from fastf1.core import Session

years = [2018, 2019, 2020, 2021, 2022, 2023, 2024]

for year in years:
    try:
        schedule = fastf1.get_event_schedule(year)
        gp_names = schedule['EventName'].tolist()
    except Exception as e:
        print(f"Could not load schedule for year {year}: {e}")
        continue

    for gp in gp_names:
        if gp not in gp_names:
            print(f"Skipping {year} {gp} (not in schedule)")
            continue

        try:
            session = fastf1.get_session(year, gp, 'R')
            session.load()
            laps = session.laps.pick_drivers('HAM')
            gp = gp.replace(' ','_')
            filename = f"data/{year}_{gp}_laps.csv"
            laps.to_csv(filename)
            print(f"Saved data for {year} {gp}")
        except Exception as e:
            print(f"Skipping {year} {gp} due to error: {e}")

In [ ]:
#getting the calender for a particular year
events = fastf1.get_event_schedule(2021)
print(events[['EventName', 'EventFormat']])

In [ ]:
#getting just one session
from fastf1.core import Session
year = 2022
gp = 'Mexico City Grand Prix'
try:
        session = fastf1.get_session(year, gp, 'R')
        session.load()
        laps = session.laps.pick_drivers('HAM')
        gp = gp.replace(' ','_')
        filename = f"data/{year}_{gp}_laps.csv"
        laps.to_csv(filename)
        print(f"Saved data for {year} {gp}")
except Exception as e:
    print(f"Skipping {year} {gp} due to error: {e}")

In [5]:
df = pd.read_csv('data/2018_Abu_Dhabi_Grand_Prix_laps.csv')
print(df)

    Unnamed: 0                    Time Driver  DriverNumber  \
0          637  0 days 00:35:49.270000    HAM            44   
1          638  0 days 00:38:37.621000    HAM            44   
2          639  0 days 00:41:13.586000    HAM            44   
3          640  0 days 00:43:57.834000    HAM            44   
4          641  0 days 00:45:41.816000    HAM            44   
5          642  0 days 00:47:26.502000    HAM            44   
6          643  0 days 00:49:34.886000    HAM            44   
7          644  0 days 00:52:02.348000    HAM            44   
8          645  0 days 00:53:47.243000    HAM            44   
9          646  0 days 00:55:32.387000    HAM            44   
10         647  0 days 00:57:17.311000    HAM            44   
11         648  0 days 00:59:02.503000    HAM            44   
12         649  0 days 01:00:47.524000    HAM            44   
13         650  0 days 01:02:32.291000    HAM            44   
14         651  0 days 01:04:17.654000    HAM          